In [248]:
import numpy as np
import pandas as pd
from pandas.io.formats.style import Styler
import dataframe_image as dfi

In [249]:
## Constants
# Ambient Conditions
h = 0  # km
P_a = 101326  # N/m^2 (std)
T_a = 288.15  # K (std)
a_a = 340.3  # m/s (std)
rho_a = 1.225  # kg/m^3

# Universal Constanrs
R = 287  # J/kg-K
gamma_c = 1.4
gamma_h = 1.333
cpa = 1.005 * 1000  # J/kg-K
cpg = 1.148 * 1000  # J/kg-K

# Operating Conditions
M_0 = 0

## Specifications
# General
mdot_a = 520  # kg/s

# Fan
BPR = 7.0
pi_f = 1.6
eta_pf = 0.91

# Compressor
eta_pc = 0.9  # MINIMUM
pi_c = 18.0

# Turbine
TIT = 2150  # K
eta_pt = 0.93
phi_t = 0.75  # MINIMUM
psi_t = 3.0  # MAXIMUM

# Criterion
DE_HALLER_MIN = 0.65
TIP_MACH_MAX = 1.2

## Preliminary Calculations
mdot_h = mdot_a/(BPR+1)
mdot_c = (mdot_a*BPR)/(BPR+1)

In [250]:
## Variables
C_a = 150  # m/s [150, 200]
t_h_rat = 0.3  # [0, 1]
M_tip = 1.1  # [0, 1.2]
lam = 1
dehaller = 0.764

In [251]:
## Initial Conditions
T_t1 = T_a
P_t1 = P_a


T_1 = T_t1 - ((C_a**2)/(2*cpa))
P_1 = P_t1*((T_1/T_t1)**(gamma_c/(gamma_c-1)))
rho_1 = P_1/(R*T_1)
a_1 = np.sqrt(gamma_c*R*T_1)

## Fan
P_t2 = P_t1*pi_f
T_t2 = T_t1 + T_t1*((pi_f**((gamma_c-1)/(gamma_c*eta_pf)))-1)

T_2 = T_t2 - ((C_a**2)/(2*cpa))
P_2 = P_t2*((T_2/T_t2)**(gamma_c/(gamma_c-1)))

rho_2 = P_2/(R*T_2)
a_2 = np.sqrt(gamma_c*R*T_2)

## Compressor
P_t3 = pi_c*P_t2
T_t3 = T_t2*((pi_c**((gamma_c-1)/(gamma_c*eta_pc)))-1) + T_t2

T_3 = T_t3 - ((C_a**2)/(2*cpa))
P_3 = P_t3*((T_3/T_t3)**(gamma_c/(gamma_c-1)))

rho_3 = P_3/(R*T_3)

A_ec = mdot_h/(rho_3*C_a)
w_c = cpa*(T_t3 - T_t2)

## Design Process

In [252]:
r_ft = np.sqrt(mdot_a/(np.pi*rho_1*C_a*(1-t_h_rat**2)))   # fan tip radius
A_t = np.pi*r_ft**2                                     # total area
A_c = A_t/(BPR+1)   
r_ch = r_ft * t_h_rat                                   # fan hub radius
r_ct = np.sqrt(A_c/(2*np.pi) + r_ch**2)                 # compressor tip radius
ht_rat_c = r_ch/r_ct                                    # compressor hub-to-tip ratio
r_m = (r_ct + r_ch)/2 

print(r_ft)

## Compressor
U_ct = M_tip * a_2
N = U_ct/(2*np.pi*r_ct)

# Mean line
U_m = 2*np.pi*N*r_m
beta_1m = np.arctan(U_m/C_a)
V_2 = C_a/np.cos(beta_1m)
V_3 = dehaller* V_2
beta_2m = np.arccos(C_a/V_3)

delta_T_stage = lam*U_m*C_a*(np.tan(beta_1m) - np.tan(beta_2m))/cpa
num_stages = (T_t3 - T_t2)/delta_T_stage

# Compressor Tip
U_t = U_ct
beta_1t = np.arctan(U_t/C_a)
V_2t = C_a/np.cos(beta_1t)
V_3t = dehaller*V_2t
beta_2t = np.arccos(C_a/V_3t)

# Compressor Root
U_r = 2*np.pi*N*r_ch
beta_1r = np.arctan(U_r/C_a)
V_2r = C_a/np.cos(beta_1r)
V_3r = dehaller * V_2r
beta_2r = np.arccos(C_a/V_3r)

# Exit Dimensions
h_ec = A_ec/(2*np.pi*r_m)
r_tec = r_m + h_ec/2
r_rec = r_m - h_ec/2

print("\nTemperature change per stage:",np.real(np.round(delta_T_stage, 2)),"K")
print("Total change in temp across compressor:", np.real(np.round(T_t3 - T_t2, 2)))
print("Number of stages:", int(np.real(num_stages)+1))

print("\n\nCompressor Blade Angles:")

print("\n  Tip Line")
print("   Beta 1:", np.real(np.round(beta_1t*180/np.pi, 2)), "degrees")
print("   Beta 2:", np.real(np.round(beta_2t*180/np.pi, 2)), "degrees")

print("\n  Mean Line")
print("   Beta 1:", np.real(np.round(beta_1m*180/np.pi, 2)), "degrees")
print("   Beta 2:", np.real(np.round(beta_2m*180/np.pi, 2)), "degrees")

print("\n  Root Line")
print("   Beta 1:", np.real(np.round(beta_1r*180/np.pi, 2)), "degrees")
print("   Beta 2:", np.real(np.round(beta_2r*180/np.pi, 2)), "degrees")

1.0453460595973854

Temperature change per stage: 35.12 K
Total change in temp across compressor: 502.03
Number of stages: 15


Compressor Blade Angles:

  Tip Line
   Beta 1: 69.26 degrees
   Beta 2: 62.39 degrees

  Mean Line
   Beta 1: 66.82 degrees
   Beta 2: 58.98 degrees

  Root Line
   Beta 1: 63.76 degrees
   Beta 2: 54.64 degrees


In [253]:
def signif(x, p):
    x = np.asarray(x)
    x_positive = np.where(np.isfinite(x) & (x != 0), np.abs(x), 10**(p-1))
    mags = 10 ** (p - 1 - np.floor(np.log10(x_positive)))
    return np.round(x * mags) / mags

In [254]:
num_stages = int(np.real(num_stages)+1)

"""
Variables to keep track of:
     0: Total Pressure
     1: Total Temperature
     2: Static Pressure
     3: Static Temperature
     4: Density
     5: Area
     6: Root Radius
     7: Tip Radius
     8: Root Alpha 1
     9: Root Beta 1
    10: Root Alpha 2
    11: Root Beta 2
    12: Root Alpha 3
    13: Root Beta 3
    14: Root de Haller Criterion
    15: Tip Alpha 1
    16: Tip Beta 1
    17: Tip Alpha 2
    18: Tip Beta 2
    19: Tip Alpha 3
    20: Tip Beta 3
    21: Tip de Haller Criterion
"""

label_array = [
    "Total Pressure (kPa)", 
    "Total Temperature (K)", 
    "Static Pressure (kPa)", 
    "Static Temperature (K)", 
    "Density (N/m^3)", 
    "Area (m^2)", 
    "Root Radius (m)", 
    "Tip Radius (m)", 
    "Root Alpha 1 (deg)", 
    "Root Beta 1 (deg)", 
    "Root Alpha 2 (deg)", 
    "Root Beta 2 (deg)", 
    "Root Alpha 3 (deg)", 
    "Root Beta 3 (deg)", 
    "Root de Haller Criterion", 
    "Tip Alpha 1 (deg)", 
    "Tip Beta 1 (deg)", 
    "Tip Alpha 2 (deg)", 
    "Tip Beta 2 (deg)", 
    "Tip Alpha 3 (deg)", 
    "Tip Beta 3 (deg)", 
    "Tip de Haller Criterion"
    ]

decimal_array = [
    '{:.1f}',
    '{:.1f}',
    '{:.1f}',
    '{:.1f}',
    '{:.2f}',
    '{:.3f}',
    '{:.3f}',
    '{:.3f}',
    '{:.0f}',
    '{:.0f}',
    '{:.0f}',
    '{:.0f}',
    '{:.0f}',
    '{:.0f}',
    '{:.3f}',    
    '{:.0f}',
    '{:.0f}',
    '{:.0f}',
    '{:.0f}',
    '{:.0f}',
    '{:.0f}',
    '{:.3f}'
]

style_dict = dict(zip(label_array, decimal_array))

variable_array = np.zeros((num_stages, 22))

## Populate initial conditions
initial_conditions = np.array([
    [P_t2],
    [T_t2],
    [P_2],
    [T_2],
    [rho_2],
    [A_c],
    [float('NaN')],
    [float('NaN')],
    [float('NaN')],
    [float('NaN')],
    [float('NaN')],
    [float('NaN')],
    [float('NaN')],
    [float('NaN')],
    [float('NaN')],
    [float('NaN')],
    [float('NaN')],
    [float('NaN')],
    [float('NaN')],
    [float('NaN')],
    [float('NaN')],
    [float('NaN')]
    ]).T

variable_array = np.vstack((initial_conditions, variable_array))

delta_T_stage = np.real(delta_T_stage)
variable_array = np.real(variable_array)
index_array = ['Initial Conditions']

for i in range(num_stages):
    index_array.append(f'Stage {i + 1}')

    T_ts = variable_array[i, 1] + delta_T_stage
    P_ts = variable_array[i, 0] * (delta_T_stage/variable_array[i, 1]+1)**((gamma_c*eta_pc)/(gamma_c-1))

    Ps = P_ts - C_a**2/(2*cpa)
    Ts = T_ts - C_a**2/(2*cpa)

    rho_s = Ps/(R*Ts)
    As = mdot_c/(rho_s*C_a)

    h_s = (As*N)/U_m
    r_ts = r_m + (h_s/2)
    r_rs = r_m - (h_s/2)

    U_ts = 2*np.pi*N*r_ts
    U_rs = 2*np.pi*N*r_rs

    beta_1rs = np.arctan(U_rs/C_a)
    V_1rs = C_a/np.cos(beta_1rs)
    V_2rs = dehaller * V_1rs
    beta_2rs = np.arctan(C_a/V_2rs)
    beta_3rs = beta_1rs

    alpha_1rs = 0.0
    # C_w2rs = U_rs - V_2rs*np.sin(beta_2rs)
    # alpha_2rs = np.arctan(C_w2rs/C_a)
    alpha_2rs = np.arctan((cpa*delta_T_stage)/(U_rs*C_a))
    alpha_3rs = alpha_1rs

    beta_1ts = np.arctan(U_ts/C_a)
    V_1ts = C_a/np.cos(beta_1ts)
    V_2ts = dehaller * V_1ts
    beta_2ts = np.arctan(C_a/V_2ts)
    beta_3ts = beta_1ts

    alpha_1ts = 0.0
    # C_w2ts = U_ts - V_2ts*np.sin(beta_2ts)
    # alpha_2ts = np.arctan(C_w2ts/C_a)
    alpha_2ts = np.arctan((cpa*delta_T_stage)/(U_ts*C_a))
    alpha_3ts = alpha_1ts

    dehaller_rs = V_2rs/V_1rs
    dehaller_ts = V_2ts/V_1ts

    alpha_1rs, alpha_2rs, alpha_3rs, beta_1rs, beta_2rs, beta_3rs = np.array([alpha_1rs, alpha_2rs, alpha_3rs, beta_1rs, beta_2rs, beta_3rs]) * 180/np.pi
    alpha_1ts, alpha_2ts, alpha_3ts, beta_1ts, beta_2ts, beta_3ts = np.array([alpha_1ts, alpha_2ts, alpha_3ts, beta_1ts, beta_2ts, beta_3ts]) * 180/np.pi

    variable_array[i+1, :] =\
         [P_ts, T_ts, Ps, Ts, rho_s, As, r_rs, r_ts, alpha_1rs, beta_1rs, alpha_2rs, beta_2rs, alpha_3rs, beta_3rs, dehaller_rs, alpha_1ts, beta_1ts, alpha_2ts, beta_2ts, alpha_3ts, beta_3ts, dehaller_ts]

# print(np.real(np.round(variable_array)))

variable_array_sf = variable_array.copy()
variable_array_sf[:, 0] = variable_array_sf[:, 0]/1000
variable_array_sf[:, 2] = variable_array_sf[:, 2]/1000

df = pd.DataFrame(variable_array_sf, columns=label_array, index=index_array)
df_styled = df.style.format(style_dict)
display(df_styled)
dfi.export(df_styled, 'Compressor-Table.png')

0.42912128704857144


,Total Pressure (kPa),Total Temperature (K),Static Pressure (kPa),Static Temperature (K),Density (N/m^3),Area (m^2),Root Radius (m),Tip Radius (m),Root Alpha 1 (deg),Root Beta 1 (deg),Root Alpha 2 (deg),Root Beta 2 (deg),Root Alpha 3 (deg),Root Beta 3 (deg),Root de Haller Criterion,Tip Alpha 1 (deg),Tip Beta 1 (deg),Tip Alpha 2 (deg),Tip Beta 2 (deg),Tip Alpha 3 (deg),Tip Beta 3 (deg),Tip de Haller Criterion
Initial Conditions,162.1,334.0,143.9,322.8,1.55,0.429,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Stage 1,222.1,369.1,222.1,357.9,2.16,1.403,0.052,0.670,0,18,78,51,0,18,0.764,0,77,20,16,0,77,0.764
Stage 2,295.8,404.2,295.8,393.0,2.62,1.157,0.106,0.616,0,34,66,47,0,34,0.764,0,76,21,18,0,76,0.764
Stage 3,384.5,439.3,384.5,428.1,3.13,0.969,0.147,0.575,0,44,59,43,0,44,0.764,0,75,23,19,0,75,0.764
Stage 4,489.9,474.4,489.9,463.2,3.68,0.823,0.179,0.542,0,49,54,41,0,49,0.764,0,74,24,20,0,74,0.764
Stage 5,613.5,509.6,613.5,498.4,4.29,0.707,0.205,0.517,0,53,50,38,0,53,0.764,0,73,25,21,0,73,0.764
Stage 6,756.8,544.7,756.8,533.5,4.94,0.614,0.226,0.496,0,56,47,36,0,56,0.764,0,73,26,21,0,73,0.764
Stage 7,921.5,579.8,921.4,568.6,5.65,0.537,0.242,0.479,0,57,45,35,0,57,0.764,0,72,27,22,0,72,0.764
Stage 8,1109.0,614.9,1109.0,603.7,6.40,0.474,0.256,0.465,0,59,43,34,0,59,0.764,0,72,28,22,0,72,0.764
Stage 9,1321.0,650.0,1321.0,638.8,7.20,0.421,0.268,0.454,0,60,42,33,0,60,0.764,0,71,28,23,0,71,0.764
